In [1]:
import pandas as pd

In [2]:
data1=pd.read_csv("patents_not_found.csv")

In [3]:
data2=pd.read_csv("patents_without_embeddings.csv")

In [4]:
data1.head()

,key
0,EP470080A1
1,EP0823839A1
2,EP221816A2
3,EP332113A2
4,EP0519036A1


In [5]:
data2.head()

,key
0,EP0286634A1
1,EP0320505A1
2,EP0401206A1
3,EP0670744A1
4,EP0607158A1


In [6]:
len(data1)

11487

In [7]:
len(data2)

85153

In [8]:
data = pd.concat([data1, data2], axis=0, ignore_index=True)

In [9]:
data.head()

,key
0,EP470080A1
1,EP0823839A1
2,EP221816A2
3,EP332113A2
4,EP0519036A1


In [10]:
len(data)

96640

In [11]:
import mysql.connector
import os
from dotenv import load_dotenv

In [12]:
load_dotenv()
db_config = {
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_DATABASE')
}

In [13]:
#sql query to get appln_ids of representative patents with the help of keys
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
keys=tuple(data['key'])
query = f"""
SELECT 
	t1.publn_auth,
	t1.appln_id,   
    t1.publn_nr,
    t1.publn_kind
FROM
	patstat_s24.tls211_pat_publn t1
WHERE
    CONCAT(t1.publn_auth, t1.publn_nr, t1.publn_kind) IN {keys}
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [14]:
data3=pd.DataFrame(results)

In [15]:
data3.head()

,publn_auth,appln_id,publn_nr,publn_kind
0,EP,12717,1947248,A1
1,EP,48850,1955742,A1
2,EP,176206,1972749,A1
3,EP,126845,1982623,A1
4,EP,277044,2005998,A2


In [16]:
len(data3)

96640

In [17]:
data3['key']=data3['publn_auth']+data3['publn_nr']+data3['publn_kind']

In [18]:
data3.head()

,publn_auth,appln_id,publn_nr,publn_kind,key
0,EP,12717,1947248,A1,EP1947248A1
1,EP,48850,1955742,A1,EP1955742A1
2,EP,176206,1972749,A1,EP1972749A1
3,EP,126845,1982623,A1,EP1982623A1
4,EP,277044,2005998,A2,EP2005998A2


In [19]:
data3=data3[['key','appln_id']]

In [20]:
len(data3)

96640

In [21]:
data3.head()

,key,appln_id
0,EP1947248A1,12717
1,EP1955742A1,48850
2,EP1972749A1,176206
3,EP1982623A1,126845
4,EP2005998A2,277044


In [22]:
data3['appln_id'][0:5]

0     12717
1     48850
2    176206
3    126845
4    277044
Name: appln_id, dtype: int64

In [23]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
ids=tuple(data3['appln_id'])
query = f"""
SELECT 
	t1.appln_id,   
    t1.appln_title
FROM
	patstat_s24.tls202_appln_title t1
WHERE
    t1.appln_id IN {ids}
    AND t1.appln_title_lg = 'en'
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [24]:
data4=pd.DataFrame(results)

In [25]:
data4.head()

,appln_id,appln_title
0,28,DRIVE COMPRISING A MOTOR AND AT LEAST ONE GEAR...
1,92,DEVICE FOR FIXING A SKI BINDING TO A SKI
2,94,PIECE OF FURNITURE IN PARTICULAR A SEAT
3,122,REMOTE CONTROLLABLE LOCKING DEVICE FOR A VEHICLE
4,128,GN CONTAINER


In [26]:
len(data4)

96640

In [27]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
ids=tuple(data3['appln_id'])
query = f"""
SELECT 
	t1.appln_id,   
    t1.appln_title
FROM
	patstat_s24.tls202_appln_title t1
WHERE
    t1.appln_id IN {ids}
    AND t1.appln_title_lg != 'en'
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [28]:
pd.DataFrame(results).head()

""


In [29]:
data5 = pd.merge(data3, data4, on='appln_id', how='inner')

In [30]:
data5.head()

,key,appln_id,appln_title
0,EP1947248A1,12717,ANTISEISMIC CONSTRUCTION
1,EP1955742A1,48850,DEVICE FOR TRAINING AND IMPROVING A VOLLEYBALL...
2,EP1972749A1,176206,Door or window comprising means for laterally ...
3,EP1982623A1,126845,COASTER FOR DRINKING VESSELS
4,EP2005998A2,277044,RADIOCOMMUNICATION SYSTEM FOR A TEAM SPORT GAME


In [31]:
len(data5)

96640

In [32]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
ids=tuple(data5['appln_id'])
query = f"""
SELECT 
	t1.appln_id,   
    t1.appln_abstract
FROM
	patstat_s24.tls203_appln_abstr t1
WHERE
    t1.appln_id IN {ids}
    AND t1.appln_abstract_lg = 'en'
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [33]:
data6=pd.DataFrame(results)

In [34]:
data6.head()

,appln_id,appln_abstract
0,28,The invention relates to a drive comprising a ...
1,92,The invention relates to a device for fixing a...
2,94,"The invention relates to a piece of furniture,..."
3,122,The invention relates to a remote controlled l...
4,128,The invention relates to an intelligent GN (ga...


In [35]:
len(data6)

79516

In [37]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
ids=tuple(data5['appln_id'])
query = f"""
SELECT 
	t1.appln_id,   
    t1.appln_abstract
FROM
	patstat_s24.tls203_appln_abstr t1
WHERE
    t1.appln_id IN {ids}
    AND t1.appln_abstract_lg != 'en'
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [38]:
data7=pd.DataFrame(results)

In [39]:
data7.head()

,appln_id,appln_abstract
0,667,"Es ist ein Seil aus Fäden (2), Garnen, Zwirnen..."
1,673,Bei einer optischen Anzeige- und Funktionskont...
2,713,Eine Verbundplatte besteht aus einer Naturstei...
3,4390,Procédé de fabrication de ces compacteurs dans...
4,4675,Procédé et dispositif (40) permettant de distr...


In [40]:
len(data7)

15693

In [41]:
15693+79516

95209

In [42]:
#confirming

In [43]:
data67 = pd.concat([data6, data7], axis=0, ignore_index=True)

In [44]:
data67.head()

,appln_id,appln_abstract
0,28,The invention relates to a drive comprising a ...
1,92,The invention relates to a device for fixing a...
2,94,"The invention relates to a piece of furniture,..."
3,122,The invention relates to a remote controlled l...
4,128,The invention relates to an intelligent GN (ga...


In [45]:
len(data67)

95209

In [46]:
data67_2 = pd.merge(data5, data67, on='appln_id', how='inner')

In [47]:
data67_2.head()

,key,appln_id,appln_title,appln_abstract
0,EP1947248A1,12717,ANTISEISMIC CONSTRUCTION,Die Erfindung betrifft ein Verfahren für eine ...
1,EP1955742A1,48850,DEVICE FOR TRAINING AND IMPROVING A VOLLEYBALL...,Die Erfindung bezieht sich auf eine Einrichtun...
2,EP1972749A1,176206,Door or window comprising means for laterally ...,"Porte, fenêtre, ou analogue, à châssis formé d..."
3,EP1982623A1,126845,COASTER FOR DRINKING VESSELS,Es handelt sich um einen Untersetzer für mit F...
4,EP2005998A2,277044,RADIOCOMMUNICATION SYSTEM FOR A TEAM SPORT GAME,Die Erfindung bezieht sich auf ein Funkverbind...


In [48]:
len(data67_2)

95209

In [49]:
data67_3 = pd.merge(data5, data67, on='appln_id', how='outer', indicator=True)

In [50]:
data67_3.head()

,key,appln_id,appln_title,appln_abstract,_merge
0,EP1380090A1,28,DRIVE COMPRISING A MOTOR AND AT LEAST ONE GEAR...,The invention relates to a drive comprising a ...,both
1,EP1809396A1,92,DEVICE FOR FIXING A SKI BINDING TO A SKI,The invention relates to a device for fixing a...,both
2,EP1850699A1,94,PIECE OF FURNITURE IN PARTICULAR A SEAT,"The invention relates to a piece of furniture,...",both
3,EP1928710A1,122,REMOTE CONTROLLABLE LOCKING DEVICE FOR A VEHICLE,The invention relates to a remote controlled l...,both
4,EP1935212A1,128,GN CONTAINER,The invention relates to an intelligent GN (ga...,both


In [55]:
data67_4=data67_3[data67_3['_merge']!="both"]

In [59]:
data67_4[['key','appln_id']]

,key,appln_id
57,EP0147905A2,4379
59,EP0219694A2,4443
61,EP215657A2,4525
194,EP0214633A1,41453
286,EP0220904A2,57017
...,...,...
84420,EP2315539A2,274692831
88500,EP2365149A2,329076126
92568,EP2593936A2,336583549
92693,EP2663207A1,336706345


In [60]:
data67_4[['key','appln_id']].to_csv("applnids_notin_tls203_appln_abstr.csv",index=False)

In [61]:
pd.read_csv("applnids_notin_tls203_appln_abstr.csv").head()

,key,appln_id
0,EP0147905A2,4379
1,EP0219694A2,4443
2,EP215657A2,4525
3,EP0214633A1,41453
4,EP0220904A2,57017


In [63]:
len(pd.read_csv("applnids_notin_tls203_appln_abstr.csv"))

1431

In [64]:
#patents not in tls203_appln_abstr = 1431

In [65]:
data7.head()

,appln_id,appln_abstract
0,667,"Es ist ein Seil aus Fäden (2), Garnen, Zwirnen..."
1,673,Bei einer optischen Anzeige- und Funktionskont...
2,713,Eine Verbundplatte besteht aus einer Naturstei...
3,4390,Procédé de fabrication de ces compacteurs dans...
4,4675,Procédé et dispositif (40) permettant de distr...


In [66]:
len(data7)

15693

In [67]:
data_not_en = pd.merge(data5, data7, on='appln_id', how='inner')

In [68]:
data_not_en.head()

,key,appln_id,appln_title,appln_abstract
0,EP1947248A1,12717,ANTISEISMIC CONSTRUCTION,Die Erfindung betrifft ein Verfahren für eine ...
1,EP1955742A1,48850,DEVICE FOR TRAINING AND IMPROVING A VOLLEYBALL...,Die Erfindung bezieht sich auf eine Einrichtun...
2,EP1972749A1,176206,Door or window comprising means for laterally ...,"Porte, fenêtre, ou analogue, à châssis formé d..."
3,EP1982623A1,126845,COASTER FOR DRINKING VESSELS,Es handelt sich um einen Untersetzer für mit F...
4,EP2005998A2,277044,RADIOCOMMUNICATION SYSTEM FOR A TEAM SPORT GAME,Die Erfindung bezieht sich auf ein Funkverbind...


In [69]:
len(data_not_en)

15693

In [77]:
data_not_en[['key','appln_id']].to_csv("patents_not_en_abstract.csv",index=False)

In [78]:
pd.read_csv("patents_not_en_abstract.csv").head()

,key,appln_id
0,EP1947248A1,12717
1,EP1955742A1,48850
2,EP1972749A1,176206
3,EP1982623A1,126845
4,EP2005998A2,277044


In [72]:
#patents with non english abstract=15693

In [73]:
data_with_en = pd.merge(data5, data6, on='appln_id', how='inner')

In [74]:
data_with_en.head()

,key,appln_id,appln_title,appln_abstract
0,EP2074909A1,56455537,Extending roller guide,The guide has an insertion unit (25) with a st...
1,EP2040575A1,55625363,INSOLE,An insole or foot-strap for footwear that comp...
2,EP1941003A2,352951,METHODS OF FILTERING A LIQUID STREAM PRODUCED ...,The invention provides methods for producing o...
3,EP1966449A1,2268,CONCRETE FLOOR DEVICE,The present invention relates to a concrete fl...
4,EP1934419A1,16369679,FABRIC SHELTER,The invention concerns a shelter comprising a ...


In [75]:
len(data_with_en)

79516

In [ ]:
#data_with_engilsh abstact and title=79516

In [76]:
79516+15693+1431

96640

In [ ]:
#data_with_engilsh abstact and title=79516
#patents with non english abstract=15693
#patents not in tls203_appln_abstr = 1431

In [ ]:
data_with_en.to_csv("12english_abstract_title.csv",index=False)